In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [46]:
data = pd.read_csv("labeled_dataset.csv")
#data.drop('', inplace=True, axis=1)

In [47]:
dataset = pd.read_csv("data-final.csv",delimiter="\t")
print('Is there any missing value? ', dataset.isnull().values.any())
print('How many missing values? ', dataset.isnull().values.sum())
dataset.dropna(inplace=True)
print('Number of participants after eliminating missing values: ', len(dataset))
dataset.columns

Is there any missing value?  True
How many missing values?  186358
Number of participants after eliminating missing values:  1012050


Index(['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9',
       'EXT10',
       ...
       'dateload', 'screenw', 'screenh', 'introelapse', 'testelapse',
       'endelapse', 'IPC', 'country', 'lat_appx_lots_of_err',
       'long_appx_lots_of_err'],
      dtype='object', length=110)

In [48]:
keepColumns = ['EXT1', 'EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9', 'EXT10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7', 'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6', 'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8', 'EST9', 'EST10']
dataset = dataset[[col for col in dataset.columns if col in keepColumns]]
print(dataset)

         EXT1  EXT2  EXT3  EXT4  EXT5  EXT6  EXT7  EXT8  EXT9  EXT10  ...  \
0         4.0   1.0   5.0   2.0   5.0   1.0   5.0   2.0   4.0    1.0  ...   
1         3.0   5.0   3.0   4.0   3.0   3.0   2.0   5.0   1.0    5.0  ...   
2         2.0   3.0   4.0   4.0   3.0   2.0   1.0   3.0   2.0    5.0  ...   
3         2.0   2.0   2.0   3.0   4.0   2.0   2.0   4.0   1.0    4.0  ...   
4         3.0   3.0   3.0   3.0   5.0   3.0   3.0   5.0   3.0    4.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...    ...  ...   
1015336   4.0   2.0   4.0   3.0   4.0   3.0   3.0   3.0   3.0    3.0  ...   
1015337   4.0   3.0   4.0   3.0   3.0   3.0   4.0   4.0   3.0    3.0  ...   
1015338   4.0   2.0   4.0   3.0   5.0   1.0   4.0   2.0   4.0    4.0  ...   
1015339   2.0   4.0   3.0   4.0   2.0   2.0   1.0   4.0   2.0    4.0  ...   
1015340   4.0   2.0   4.0   2.0   4.0   1.0   4.0   2.0   4.0    4.0  ...   

         OPN1  OPN2  OPN3  OPN4  OPN5  OPN6  OPN7  OPN8  OPN9  OPN10  
0   

In [49]:
# Groups and Questions
ext_questions = {'EXT1' : 'I am the life of the party',
                 'EXT2' : 'I dont talk a lot',
                 'EXT3' : 'I feel comfortable around people',
                 'EXT4' : 'I keep in the background',
                 'EXT5' : 'I start conversations',
                 'EXT6' : 'I have little to say',
                 'EXT7' : 'I talk to a lot of different people at parties',
                 'EXT8' : 'I dont like to draw attention to myself',
                 'EXT9' : 'I dont mind being the center of attention',
                 'EXT10': 'I am quiet around strangers'}

est_questions = {'EST1' : 'I get stressed out easily',
                 'EST2' : 'I am relaxed most of the time',
                 'EST3' : 'I worry about things',
                 'EST4' : 'I seldom feel blue',
                 'EST5' : 'I am easily disturbed',
                 'EST6' : 'I get upset easily',
                 'EST7' : 'I change my mood a lot',
                 'EST8' : 'I have frequent mood swings',
                 'EST9' : 'I get irritated easily',
                 'EST10': 'I often feel blue'}

agr_questions = {'AGR1' : 'I feel little concern for others',
                 'AGR2' : 'I am interested in people',
                 'AGR3' : 'I insult people',
                 'AGR4' : 'I sympathize with others feelings',
                 'AGR5' : 'I am not interested in other peoples problems',
                 'AGR6' : 'I have a soft heart',
                 'AGR7' : 'I am not really interested in others',
                 'AGR8' : 'I take time out for others',
                 'AGR9' : 'I feel others emotions',
                 'AGR10': 'I make people feel at ease'}

csn_questions = {'CSN1' : 'I am always prepared',
                 'CSN2' : 'I leave my belongings around',
                 'CSN3' : 'I pay attention to details',
                 'CSN4' : 'I make a mess of things',
                 'CSN5' : 'I get chores done right away',
                 'CSN6' : 'I often forget to put things back in their proper place',
                 'CSN7' : 'I like order',
                 'CSN8' : 'I shirk my duties',
                 'CSN9' : 'I follow a schedule',
                 'CSN10' : 'I am exacting in my work'}

opn_questions = {'OPN1' : 'I have a rich vocabulary',
                 'OPN2' : 'I have difficulty understanding abstract ideas',
                 'OPN3' : 'I have a vivid imagination',
                 'OPN4' : 'I am not interested in abstract ideas',
                 'OPN5' : 'I have excellent ideas',
                 'OPN6' : 'I do not have a good imagination',
                 'OPN7' : 'I am quick to understand things',
                 'OPN8' : 'I use difficult words',
                 'OPN9' : 'I spend time reflecting on things',
                 'OPN10': 'I am full of ideas'}


In [50]:
og_cols = data.columns.tolist()
og_cols[:50] = keepColumns
data.columns = og_cols
data = data[:22]
print(data)
data.to_csv("modified.csv", index=False)


    EXT1  EXT2  EXT3  EXT4  EXT5  EXT6  EXT7  EXT8  EXT9  EXT10  ...  EST5  \
0    3.0   3.0   4.0   2.0   4.0   3.0   3.0   3.0   3.0    2.0  ...   3.0   
1    5.0   1.0   5.0   1.0   5.0   1.0   5.0   1.0   5.0    1.0  ...   5.0   
2    3.0   2.0   3.0   2.0   3.0   3.0   3.0   3.0   3.0    3.0  ...   4.0   
3    3.0   4.0   4.0   3.0   3.0   3.0   3.0   3.0   3.0    3.0  ...   3.0   
4    5.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0   3.0    3.0  ...   3.0   
5    3.0   2.0   3.0   3.0   2.0   2.0   2.0   2.0   2.0    2.0  ...   4.0   
6    4.0   4.0   4.0   2.0   4.0   3.0   4.0   4.0   4.0    4.0  ...   4.0   
7    3.0   2.0   4.0   1.0   3.0   3.0   3.0   3.0   3.0    3.0  ...   3.0   
8    3.0   2.0   4.0   3.0   3.0   3.0   4.0   3.0   3.0    3.0  ...   4.0   
9    3.0   2.0   5.0   2.0   3.0   2.0   3.0   3.0   3.0    3.0  ...   3.0   
10   5.0   1.0   5.0   1.0   5.0   1.0   5.0   3.0   3.0    3.0  ...   5.0   
11   3.0   3.0   2.0   4.0   4.0   3.0   3.0   3.0   3.0    3.0 

In [95]:
import pandas as pd

# Assuming 'dataset' and 'data' are DataFrames

subset = dataset.iloc[:50000, :].copy()

# Select features for clustering (assuming no "Job Domain:" column initially)
features = subset

# Perform KMeans clustering
kmeans = KMeans(n_clusters=len(data["Job Domain:"])).fit(features)
indexes = list(kmeans.labels_);
#print(indexes)


# Map cluster labels to original labels from the 'data' DataFrame
label_mapping = dict(zip(kmeans.labels_, data["Job Domain:"]))
print(label_mapping)

# Create a new column in 'subset' to store original labels
subset["Job Domain:"] = None  # Initialize with empty values

# Assign original labels using the mapping
keys_list = list(label_mapping.keys())
print(type(keys_list))
for i in range(len(subset)):
    if(indexes[i] in keys_list):
        subset.loc[i, "Job Domain:"] = label_mapping[indexes[i]]

# Print the final labels (optional)
print(subset["Job Domain:"])

# Save the final dataset
subset.to_csv("final_dataset.csv", index=False)


C:\Users\Krishna Kumar\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{4: 'Computer & Mathematics', 18: 'Computer & Mathematics', 13: 'Sales & Related.', 1: 'Management', 14: 'Management', 19: 'Education, Training, & Library', 21: 'Sales & Related.', 10: 'Transportation & Material Moving', 16: 'Life, Physical, & Social Science', 6: 'Business & Financial Operations', 8: 'Business & Financial Operations', 12: 'Education, Training, & Library', 17: 'Construction & Extraction', 0: 'Transportation & Material Moving'}
<class 'dict_keys'>
0                  Computer & Mathematics
1                  Computer & Mathematics
2                        Sales & Related.
3                              Management
4                        Sales & Related.
                       ...               
50002                                None
50003                                None
1500      Business & Financial Operations
1793     Transportation & Material Moving
4030                     Sales & Related.
Name: Job Domain:, Length: 50003, dtype: object
